# Getting Started

## Install Cobrapy Package

In [0]:
!pip install cobra



## Loading a model and inspecting it

To begin with, cobrapy comes with bundled models for _Salmonella_ and _E. coli_, as well as a "textbook" model of _E. coli_ core metabolism. To load a test model, type

In [0]:
from __future__ import print_function

import cobra
import cobra.test

# "ecoli" and "salmonella" are also valid arguments
model = cobra.test.create_test_model("textbook")

When using [Jupyter notebook](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/) this type of information is rendered as a table.

In [0]:
model

The reactions, metabolites, and genes attributes of the cobrapy model are a special type of list called a `cobra.DictList`, and each one is made up of `cobra.Reaction`, `cobra.Metabolite` and `cobra.Gene` objects respectively.

In [0]:
model.reactions

In [0]:
model.metabolites

In [0]:
model.genes

Just like a regular list, objects in the `DictList` can be retrieved by index. For example, to get the 30th reaction in the model (at index 29 because of [0-indexing](https://en.wikipedia.org/wiki/Zero-based_numbering)):

In [0]:
model.reactions[29]

Additionally, items can be retrieved by their `id` using the `DictList.get_by_id()` function. For example, to get the cytosolic atp metabolite object (the id is "atp_c"), we can do the following:

In [0]:
model.metabolites.get_by_id("atp_c")

As an added bonus, users with an interactive shell such as IPython will be able to tab-complete to list elements inside a list. While this is not recommended behavior for most code because of the possibility for characters like "-" inside ids, this is very useful while in an interactive prompt:

In [0]:
model.reactions.EX_glc__D_e.bounds

## Reactions

We will consider the reaction glucose 6-phosphate isomerase, which interconverts glucose 6-phosphate and fructose 6-phosphate. The reaction id for this reaction in our test model is PGI.

In [0]:
pgi = model.reactions.get_by_id("PGI")
pgi

We can view the full name and reaction catalyzed as strings

In [0]:
print(pgi.name)
print(pgi.reaction)

We can also view reaction upper and lower bounds. Because the `pgi.lower_bound` < 0, and `pgi.upper_bound` > 0, `pgi` is reversible.

In [0]:
print(pgi.lower_bound, "< pgi <", pgi.upper_bound)
print(pgi.reversibility)

We can also ensure the reaction is mass balanced. This function will return elements which violate mass balance. If it comes back empty, then the reaction is mass balanced.

In [0]:
pgi.check_mass_balance()

In order to add a metabolite, we pass in a `dict` with the metabolite object and its coefficient

In [0]:
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): -1})
pgi.reaction

The reaction is no longer mass balanced

In [0]:
pgi.check_mass_balance()

We can remove the metabolite, and the reaction will be balanced once again.

In [0]:
pgi.subtract_metabolites({model.metabolites.get_by_id("h_c"): -1})
print(pgi.reaction)
print(pgi.check_mass_balance())

It is also possible to build the reaction from a string. However, care must be taken when doing this to ensure reaction id's match those in the model. The direction of the arrow is also used to update the upper and lower bounds.

In [0]:
pgi.reaction = "g6p_c --> f6p_c + h_c + green_eggs + ham"

In [0]:
pgi.reaction

In [0]:
pgi.reaction = "g6p_c <=> f6p_c"
pgi.reaction

## Metabolites

We will consider cytosolic atp as our metabolite, which has the id `"atp_c"` in our test model.

In [0]:
atp = model.metabolites.get_by_id("atp_c")
atp

We can print out the metabolite name and compartment (cytosol in this case) directly as string.

In [0]:
print(atp.name)
print(atp.compartment)

We can see that ATP is a charged molecule in our model.

In [0]:
atp.charge

We can see the chemical formula for the metabolite as well.

In [0]:
print(atp.formula)

The reactions attribute gives a `frozenset` of all reactions using the given metabolite. We can use this to count the number of reactions which use atp.

In [0]:
len(atp.reactions)

A metabolite like glucose 6-phosphate will participate in fewer reactions.

In [0]:
model.metabolites.get_by_id("g6p_c").reactions

## Genes

The `gene_reaction_rule` is a boolean representation of the gene requirements for this reaction to be active as described in [Schellenberger et al 2011 Nature Protocols 6(9):1290-307](http://dx.doi.org/doi:10.1038/nprot.2011.308).

The GPR is stored as the gene_reaction_rule for a Reaction object as a string.

In [0]:
gpr = pgi.gene_reaction_rule
gpr

Corresponding gene objects also exist. These objects are tracked by the reactions itself, as well as by the model

In [0]:
pgi.genes

In [0]:
pgi_gene = model.genes.get_by_id("b4025")
pgi_gene

Each gene keeps track of the reactions it catalyzes

In [0]:
pgi_gene.reactions

Altering the gene_reaction_rule will create new gene objects if necessary and update all relationships.

In [0]:
pgi.gene_reaction_rule = "(spam or eggs)"
pgi.genes

In [0]:
pgi_gene.reactions

Newly created genes are also added to the model

In [0]:
model.genes.get_by_id("spam")

The undelete_model_genes can be used to reset a gene deletion